In [1]:
import os
import json
import pandas as pd
import traceback

In [2]:
from langchain.chat_models import ChatOpenAI

In [9]:
from dotenv import load_dotenv
load_dotenv()

True

In [10]:
KEY=os.getenv("OPENAI_API_KEY")

In [12]:
llm=ChatOpenAI(openai_api_key=KEY,model_name="gpt-3.5-turbo", temperature=0.4)

In [14]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [18]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [17]:
TEMPLATE="""
text:{text}
you are an expert mcq maker. given the above text, it is your job to \ 
create a quiz of {number} multiple choice questions for {subject} students in {tone} tone. 
make sure the questions are not repeated and check all the questions to be conforming the text as well. 
make sure to format your response like RESPONSE_JSON bellow and use it as a guide. \
ensure to make {number} mcqs.
### RESPONSE_JSON 
{response_json} 

"""

In [19]:
quiz_generation_prompt=PromptTemplate(
    input_variables=["text","number","subject","tone","response_json"],
    template=TEMPLATE
)

In [24]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

In [20]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [21]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE2)

In [22]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [25]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [26]:
file_path=r"C:\Users\victo\MCQgenerator\data.txt"

In [27]:
file_path

'C:\\Users\\victo\\MCQgenerator\\data.txt'

In [28]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [29]:
print(TEXT)

Brazilian jiu-jitsu (BJJ; Portuguese: jiu-jitsu brasileiro [ʒiw ˈʒitsu bɾaziˈlejɾu, ʒu -]) is a self-defence martial art and combat sport based on grappling, ground fighting, and submission holds. BJJ focuses on taking ones opponent down to the ground, gaining a dominant position, and using a number of techniques to force them into submission via joint locks or chokeholds.

Brazilian jiu-jitsu was first developed in 1925 by Brazilian brothers Carlos, Oswaldo, Gastão Jr., O’Brien, and Hélio Gracie, after Carlos was taught a hybrid of traditional Japanese Jujitsu and Kodokan judo by a travelling Japanese judoka, Mitsuyo Maeda, in 1917. Later on, the Gracie family developed their own self-defense system Gracie Jiu-Jitsu.

BJJ eventually came to be its own defined combat sport through the innovations, practices, and adaptation of Gracie jiu-jitsu and judo, and became an essential martial art for modern MMA.

BJJ revolves around the concept that a smaller, weaker person can successfully def

In [30]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [31]:
NUMBER=4
SUBJECT="Brasilian Jiu Jitsu"
TONE="simple"

In [32]:
#https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking

#How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )

c:\Users\victo\MCQgenerator\env\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

text:Brazilian jiu-jitsu (BJJ; Portuguese: jiu-jitsu brasileiro [ʒiw ˈʒitsu bɾaziˈlejɾu, ʒu -]) is a self-defence martial art and combat sport based on grappling, ground fighting, and submission holds. BJJ focuses on taking ones opponent down to the ground, gaining a dominant position, and using a number of techniques to force them into submission via joint locks or chokeholds.

Brazilian jiu-jitsu was first developed in 1925 by Brazilian brothers Carlos, Oswaldo, Gastão Jr., O’Brien, and Hélio Gracie, after Carlos was taught a hybrid of traditional Japanese Jujitsu and Kodokan judo by a travelling Japanese judoka, Mitsuyo Maeda, in 1917. Later on, the Gracie family developed their own self-defense system Gracie Jiu-Jitsu.

BJJ eventually came to be its own defined combat sport through the innovations, practices, and adaptation of Gracie jiu-jitsu and judo, and became an essential mar

In [33]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:5691
Prompt Tokens:5084
Completion Tokens:607
Total Cost:0.00884


In [34]:
response

{'text': 'Brazilian jiu-jitsu (BJJ; Portuguese: jiu-jitsu brasileiro [ʒiw ˈʒitsu bɾaziˈlejɾu, ʒu -]) is a self-defence martial art and combat sport based on grappling, ground fighting, and submission holds. BJJ focuses on taking ones opponent down to the ground, gaining a dominant position, and using a number of techniques to force them into submission via joint locks or chokeholds.\n\nBrazilian jiu-jitsu was first developed in 1925 by Brazilian brothers Carlos, Oswaldo, Gastão Jr., O’Brien, and Hélio Gracie, after Carlos was taught a hybrid of traditional Japanese Jujitsu and Kodokan judo by a travelling Japanese judoka, Mitsuyo Maeda, in 1917. Later on, the Gracie family developed their own self-defense system Gracie Jiu-Jitsu.\n\nBJJ eventually came to be its own defined combat sport through the innovations, practices, and adaptation of Gracie jiu-jitsu and judo, and became an essential martial art for modern MMA.\n\nBJJ revolves around the concept that a smaller, weaker person can 

In [35]:
quiz=response.get("quiz")

In [36]:
quiz=json.loads(quiz)

In [37]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [38]:
quiz_table_data

[{'MCQ': 'What year was Brazilian jiu-jitsu first developed?',
  'Choices': 'a: 1925 | b: 1917 | c: 1972 | d: 1993',
  'Correct': 'a'},
 {'MCQ': 'Who were the Brazilian brothers who first developed Brazilian jiu-jitsu?',
  'Choices': "a: Carlos and Hélio Gracie | b: Carlos and Oswaldo Gracie | c: Carlos and Gastão Gracie | d: Carlos and O'Brien Gracie",
  'Correct': 'b'},
 {'MCQ': 'What is the main focus of Brazilian jiu-jitsu?',
  'Choices': 'a: Striking techniques | b: Ground fighting and grappling | c: Kicking techniques | d: Weapon defense',
  'Correct': 'b'},
 {'MCQ': 'Who won the first, second, and fourth Ultimate Fighting Championships using Brazilian jiu-jitsu?',
  'Choices': 'a: Royce Gracie | b: Rorion Gracie | c: Carlos Gracie | d: Hélio Gracie',
  'Correct': 'a'}]

In [39]:
quiz=pd.DataFrame(quiz_table_data)

In [40]:
quiz.to_csv("bjj.csv",index=False)

In [41]:
from datetime import datetime
datetime.now().strftime('%m_%d_%Y_%H_%M_%S')

'03_03_2024_18_04_33'